**Projeto de Graduação** 

*Aluno: Victor Martins de Araújo*

*Orientador: Reinaldo Garcia Cipriano*

*Política (t,s,S)*

*Simulação para otimização de uma política de estoque baseada em estratégia mista, ponto de pedido (s), ritmo de pedido (t) e no limite superior de estoque (S)*

In [1]:
import pandas as pd
import numpy as np
from google.colab import files

In [2]:
#Definindo custo de transporte
def c(ftl,quantity):
  ftl_cost = 3100
  if quantity <= ftl:
      return ftl_cost
  elif quantity > ftl:
      num_cars = quantity//ftl
  if quantity % ftl != 0:
      num_cars += 1

  return num_cars * ftl_cost

In [3]:
#Definindo função de cálculo de custo de estoque
def h(quantity,unit_price):
 if quantity < 0:
    stock_cost = 0
 else:   
    stock_cost = quantity * unit_price * 0.5247
 return stock_cost

Estratégia Baseada no ritmo de consumo do estoque em dias e no ponto de pedido (t,s,S)

In [4]:
#Definindo função de simulação 
def simulation_formula_tsS():
    #definindo os parâmetros (tempo de simulação, lead time,estoque incicial, políticas atuais)
    T = 365
    unit_price = 1.095
    ftl = 35000

    #definindo variáveis da simulação produto 8864
    holding_costs = 0
    order_costs = 0
    total_cost = 0
    missing_count = 0
    t1 = 0
    t = 0
    order = 0
    order_count = 0
    quantity_miss = 0

    #definindo vetores para combinação de políticas
    smax = list(range(350000,500000,1000))
    smin = list(range(200000,349000,1000))
    t_temp = list(range(1,10,1))

    #definindo tabela do dataframe do output
    Resultados_tsS = pd.DataFrame(columns = ['Política','Custo Total','Custo Estoque','Custo Pedido','N Pedidos','N Faltas'])

    #início do loop para combinação das diferentes políticas no tempo T
    for s_max in smax:
        
        for s_min in smin:

            for temp in t_temp :
                hct = []
                oct = []
                tc = []
                tp = []
                tf = []
                lc = []

                for i in range(10):

                    total_cost = 0
                    order_costs = 0
                    stock = 250000 #estoque inicial
                    missing_count = 0
                    order_count = 0
                    order = 0
                    holding_costs = 0
                    quantity_miss = 0

                    #Simula a dinâmica do estoque para o período de um ano
                    for i in range(1,T):

                        holding_costs += h(stock,unit_price)

                        #gerando demanda estocástica de produção 
                        demand = np.random.normal(45118,8034)

                        #projetando estoque
                        stock -= demand

                        #falta de estoque
                        if stock < 0:
                            quantity_miss += stock - demand
                            missing_count += 1
                            stock = 0
                        
                        #simulando a chegada do material em estoque
                        if i == t1 and order > 0:
                            t1 = 0
                            stock += order
                            order = 0

                        #simulando pedido  baseado no ponto de pedido (s_min)
                        if stock < s_min and order == 0:
                            L = round(np.random.normal(4.4,1.25),0) 
                            t1 = i + L
                            order = s_max - stock
                            order_costs += c(ftl,order)
                            order_count +=1

                        #simulando pedido  baseado no rítimo de pedidos (t)
                        if t == temp and order == 0:
                            #gerando lead time estocástico
                            L = round(np.random.normal(4.4,1.25),0)
                            t1 = i + L
                            t = 0
                            order = s_max - stock
                            order_costs += c(ftl,order)
                            order_count +=1

                        #atualiza o contador de tempo  
                        t += 1

                    total_cost = holding_costs + order_costs

                    hct.append(holding_costs)
                    oct.append(order_costs)
                    tc.append(total_cost)
                    tp.append(order_count)
                    tf.append(missing_count)
                    lc.append(quantity_miss)   

                #Inserindo dados resultantes no DataFrame 
                Resultados_tsS = Resultados_tsS.append({'Política': [temp,s_min,s_max],
                                                'Custo Total': sum(tc)/len(tc),
                                                'Custo Estoque': sum(hct)/len(hct),
                                                'Custo Pedido': sum(oct)/len(oct),
                                                'N Pedidos': sum(tp)/len(tp),
                                                'N Faltas':sum(tf)/len(tf),
                                                'Qtd Faltando': sum(lc)/len(lc)},
                                                ignore_index=True)

          

    #exportar excel com dados da simulação
    output = Resultados_tsS.to_excel('Resultados_tsS_2.xlsx')

    files.download('Resultados_tsS_2.xlsx')
    return output
    return Resultados_tsS


In [5]:
#chamada da função de simulação
simulation_formula_tsS()

Streaming output truncated to the last 5000 lines.
<ipython-input-4-e4c8a8d5bea6>:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Resultados_tsS = Resultados_tsS.append({'Política': [temp,s_min,s_max],
<ipython-input-4-e4c8a8d5bea6>:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Resultados_tsS = Resultados_tsS.append({'Política': [temp,s_min,s_max],
<ipython-input-4-e4c8a8d5bea6>:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Resultados_tsS = Resultados_tsS.append({'Política': [temp,s_min,s_max],
<ipython-input-4-e4c8a8d5bea6>:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Resultados_tsS = Resultados_tsS.append({'Política

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>